In [ ]:
#This folder contains the MP4 or MP3 files that need to be processed.
input_folder = r"C:\Users\Hieu Pham\Downloads\1"

mp4 => mp3

In [ ]:
import os
from moviepy.editor import VideoFileClip

def convert_mp4_to_mp3(folder_path):
    if not os.path.isdir(folder_path):
        print("❌ Folder không tồn tại.")
        return

    for filename in os.listdir(folder_path):
        if filename.endswith(".mp4"):
            mp4_path = os.path.join(folder_path, filename)
            mp3_path = os.path.join(folder_path, os.path.splitext(filename)[0] + ".mp3")
            
            try:
                print(f"🎬 Đang chuyển: {filename}")
                video = VideoFileClip(mp4_path)
                video.audio.write_audiofile(mp3_path)
                video.close()
                print(f"✅ Hoàn tất: {mp3_path}")
            except Exception as e:
                print(f"❌ Lỗi khi chuyển {filename}: {e}")

convert_mp4_to_mp3(input_folder)

open ai mp3 to srt

In [ ]:
#setting

from dotenv import load_dotenv
import os
# Load biến môi trường nếu cần
load_dotenv()
gpt_api_key = os.environ.get("CHATGPT_API_KEY")

target_bitrate = "32k"

#Maximum length for each SRT (automatically split). If you don’t want splitting, just set a very large number.
chunk_length_ms = 30 * 60 * 1000


The next 2 code cells perform the same function.
The first version does not adjust the start time of the first subtitle.

In [ ]:
import os
import openai
import tempfile
from pydub import AudioSegment

# --- CÁC HÀM XỬ LÝ SRT (Giữ nguyên) ---

def srt_time_to_ms(t):
    """Chuyển đổi định dạng thời gian của SRT (h:m:s,ms) thành milliseconds."""
    h, m, s_ms = t.split(':')
    s, ms = s_ms.split(',')
    return (int(h) * 3600 + int(m) * 60 + int(s)) * 1000 + int(ms)

def ms_to_srt_time(ms):
    """Chuyển đổi milliseconds thành định dạng thời gian của SRT (h:m:s,ms)."""
    seconds, msec = divmod(ms, 1000)
    hours, seconds = divmod(seconds, 3600)
    minutes, seconds = divmod(seconds, 60)
    return f"{hours:02d}:{minutes:02d}:{seconds:02d},{msec:03d}"

def adjust_srt_segment(srt_text, index_offset, time_offset_ms):
    """Điều chỉnh chỉ số và thời gian của một đoạn phụ đề SRT."""
    segments = srt_text.strip().split('\n\n')
    adjusted_segments = []
    
    for seg in segments:
        lines = seg.splitlines()
        if len(lines) < 3:
            continue
        
        try:
            original_index = int(lines[0])
        except ValueError:
            continue
        new_index = index_offset + original_index
        
        time_line = lines[1]
        try:
            start_time_str, _, end_time_str = time_line.split()
        except ValueError:
            continue
        
        start_ms = srt_time_to_ms(start_time_str) + time_offset_ms
        end_ms   = srt_time_to_ms(end_time_str) + time_offset_ms
        new_time_line = f"{ms_to_srt_time(start_ms)} --> {ms_to_srt_time(end_ms)}"
        
        new_segment = f"{new_index}\n{new_time_line}\n" + "\n".join(lines[2:])
        adjusted_segments.append(new_segment)
    
    return "\n\n".join(adjusted_segments)

# --- HÀM XỬ LÝ AUDIO (Giữ nguyên) ---

def process_long_audio(file_path, 
                       chunk_length_ms=45 * 1000,
                       target_bitrate="32k"
                      ):
    """
    Cắt file audio dài thành các đoạn nhỏ, gửi đến Whisper API và ghép kết quả SRT lại.
    """
    audio = AudioSegment.from_mp3(file_path)
    duration_ms = len(audio)
    print(f"Total duration: {duration_ms / 1000:.2f} seconds")
    
    srt_results = []
    global_index_offset = 0
    
    for start_ms in range(0, duration_ms, chunk_length_ms):
        end_ms = min(start_ms + chunk_length_ms, duration_ms)
        print(f"Processing chunk: {start_ms // 1000}s to {end_ms // 1000}s")
        chunk_audio = audio[start_ms:end_ms]
        
        with tempfile.NamedTemporaryFile(suffix=".mp3", delete=False) as tmp:
            chunk_file_path = tmp.name
        chunk_audio.export(chunk_file_path, format="mp3", bitrate=target_bitrate)
        
        size_bytes = os.path.getsize(chunk_file_path)
        print(f"  --> Exported chunk size: {size_bytes / (1024*1024):.2f} MB")
        
        if size_bytes > 25 * 1024 * 1024:
            os.remove(chunk_file_path)
            raise Exception(f"Chunk is too large ({size_bytes} bytes) even at {target_bitrate}. "
                            "Try reducing chunk_length_ms.")
        
        with open(chunk_file_path, "rb") as audio_file:
            transcript = openai.Audio.transcribe(
                file=audio_file,
                model="whisper-1",
                response_format="srt"
            )
        
        os.remove(chunk_file_path)
        
        adjusted_transcript = adjust_srt_segment(transcript, global_index_offset, start_ms)
        srt_results.append(adjusted_transcript)
        
        num_segments = len([seg for seg in transcript.strip().split('\n\n') if seg.strip() != ""])
        global_index_offset += num_segments
        
        output_dir = os.path.dirname(file_path)
        base_name = os.path.splitext(os.path.basename(file_path))[0]
        output_chunk_file = os.path.join(output_dir, f"{base_name}_chunk_{start_ms//1000}s.srt")
        with open(output_chunk_file, "w", encoding="utf-8") as f:
            f.write(adjusted_transcript)
        print(f"  --> Saved chunk SRT to {output_chunk_file}")

    combined_srt = "\n\n".join(srt_results)
    return combined_srt


# ===== PHẦN THỰC THI CHÍNH =====
if not gpt_api_key:
    print("Lỗi: Biến môi trường CHATGPT_API_KEY chưa được thiết lập.")
else:
    openai.api_key = gpt_api_key
    print(f"Scanning for .mp3 files in: {input_folder}")

    # Lặp qua tất cả các file trong thư mục đầu vào
    for filename in os.listdir(input_folder):
        if filename.lower().endswith(".mp3"):
            file_path = os.path.join(input_folder, filename)
            print(f"\n{'='*50}\nProcessing file: {file_path}\n{'='*50}")

            try:
                # Gọi hàm xử lý cho từng file audio
                final_srt = process_long_audio(
                    file_path,
                    chunk_length_ms=chunk_length_ms,
                    target_bitrate=target_bitrate
                )

                # Lưu file SRT cuối cùng vào cùng thư mục
                base_name = os.path.splitext(filename)[0]
                output_file = os.path.join(input_folder, f"{base_name}.srt")
                with open(output_file, "w", encoding="utf-8") as f:
                    f.write(final_srt)
                print(f"\nSUCCESS: Full transcription for '{filename}' saved to {output_file}")

            except Exception as e:
                print(f"\nERROR processing file {filename}: {e}")
                print("Skipping to the next file.")
                continue # Bỏ qua file này và tiếp tục với file tiếp theo

    print(f"\n{'='*50}\nAll .mp3 files have been processed.\n{'='*50}")

The second version has an adjustment for the start time of the first subtitle (needs re-checking).

In [ ]:
import os
import openai
import tempfile
from pydub import AudioSegment
import re # Sử dụng để tách các dòng trong SRT

# --- CÁC HÀM XỬ LÝ SRT (Giữ nguyên từ code gốc của bạn) ---
def srt_time_to_ms(t):
    """Chuyển đổi định dạng thời gian của SRT (h:m:s,ms) thành milliseconds."""
    h, m, s_ms = t.split(':')
    s, ms = s_ms.split(',')
    return (int(h) * 3600 + int(m) * 60 + int(s)) * 1000 + int(ms)

def ms_to_srt_time(ms):
    """Chuyển đổi milliseconds thành định dạng thời gian của SRT (h:m:s,ms)."""
    seconds, msec = divmod(ms, 1000)
    hours, seconds = divmod(seconds, 3600)
    minutes, seconds = divmod(seconds, 60)
    return f"{hours:02d}:{minutes:02d}:{seconds:02d},{msec:03d}"

def adjust_srt_segment(srt_text, index_offset, time_offset_ms):
    """Điều chỉnh chỉ số và thời gian của một đoạn phụ đề SRT."""
    # (Hàm này giữ nguyên, không cần thay đổi)
    segments = srt_text.strip().split('\n\n')
    adjusted_segments = []
    for seg in segments:
        lines = seg.splitlines()
        if len(lines) < 3: continue
        try: original_index = int(lines[0])
        except ValueError: continue
        new_index = index_offset + original_index
        time_line = lines[1]
        try: start_time_str, _, end_time_str = time_line.split()
        except ValueError: continue
        start_ms = srt_time_to_ms(start_time_str) + time_offset_ms
        end_ms   = srt_time_to_ms(end_time_str) + time_offset_ms
        new_time_line = f"{ms_to_srt_time(start_ms)} --> {ms_to_srt_time(end_ms)}"
        new_segment = f"{new_index}\n{new_time_line}\n" + "\n".join(lines[2:])
        adjusted_segments.append(new_segment)
    return "\n\n".join(adjusted_segments)

# MỚI: Hàm ước lượng thời lượng của text theo yêu cầu
def estimate_duration_ms(text, chars_per_second=15):
    """
    Ước tính thời gian (ms) cần thiết để đọc một đoạn văn bản.
    Mặc định tốc độ đọc là 15 ký tự/giây.
    """
    if not text:
        return 0
    num_chars = len(text)
    estimated_seconds = num_chars / chars_per_second
    return int(estimated_seconds * 1000)

# --- HÀM XỬ LÝ AUDIO (Quay về code gốc và thêm logic mới) ---

def process_long_audio(file_path, 
                       chunk_length_ms=45 * 1000,
                       target_bitrate="32k"
                      ):
    """
    Cắt file audio dài thành các đoạn nhỏ, gửi đến Whisper API và ghép kết quả SRT lại.
    THAY ĐỔI: Can thiệp vào phụ đề đầu tiên để tính toán lại start_ms.
    """
    audio = AudioSegment.from_mp3(file_path)
    duration_ms = len(audio)
    print(f"Total duration: {duration_ms / 1000:.2f} seconds")
    
    srt_results = []
    global_index_offset = 0
    
    for start_ms in range(0, duration_ms, chunk_length_ms):
        end_ms = min(start_ms + chunk_length_ms, duration_ms)
        print(f"Processing chunk: {start_ms // 1000}s to {end_ms // 1000}s")
        chunk_audio = audio[start_ms:end_ms]
        
        with tempfile.NamedTemporaryFile(suffix=".mp3", delete=False) as tmp:
            chunk_file_path = tmp.name
        chunk_audio.export(chunk_file_path, format="mp3", bitrate=target_bitrate)
        
        size_bytes = os.path.getsize(chunk_file_path)
        print(f"  --> Exported chunk size: {size_bytes / (1024*1024):.2f} MB")
        
        if size_bytes > 25 * 1024 * 1024:
            os.remove(chunk_file_path)
            raise Exception(f"Chunk is too large ({size_bytes} bytes).")
        
        with open(chunk_file_path, "rb") as audio_file:
            transcript = openai.Audio.transcribe(
                file=audio_file,
                model="whisper-1",
                response_format="srt"
            )
        
        os.remove(chunk_file_path)

        # ===== LOGIC MỚI ĐƯỢC THÊM VÀO =====
        # Chỉ can thiệp vào kết quả của chunk đầu tiên
        is_first_chunk = (start_ms == 0)
        if is_first_chunk and transcript.strip():
            print("  --> First chunk detected. Adjusting timestamp of the first subtitle...")
            # Tách các khối phụ đề
            segments = transcript.strip().split('\n\n')
            if segments:
                first_segment_lines = segments[0].splitlines()
                
                # Phân tích khối phụ đề đầu tiên
                if len(first_segment_lines) >= 3:
                    time_line = first_segment_lines[1]
                    text_lines = first_segment_lines[2:]
                    
                    try:
                        # Lấy thời gian kết thúc và nội dung text
                        _, _, end_time_str = time_line.split()
                        first_sub_end_ms = srt_time_to_ms(end_time_str)
                        first_sub_text = " ".join(text_lines)
                        
                        # Ước tính thời lượng và tính toán start_ms mới
                        estimated_dur_ms = estimate_duration_ms(first_sub_text)
                        new_start_ms = first_sub_end_ms - estimated_dur_ms - 100 # Thêm buffer 100ms
                        new_start_ms = max(0, new_start_ms) # Đảm bảo không bị âm
                        
                        print(f"    - Original End: {first_sub_end_ms}ms, Text: \"{first_sub_text[:30]}...\"")
                        print(f"    - Estimated Duration: {estimated_dur_ms}ms")
                        print(f"    - New Calculated Start: {new_start_ms}ms")
                        
                        # Tạo lại dòng thời gian và cập nhật vào khối phụ đề
                        new_start_time_str = ms_to_srt_time(new_start_ms)
                        new_time_line = f"{new_start_time_str} --> {end_time_str}"
                        first_segment_lines[1] = new_time_line
                        
                        # Ghép lại khối phụ đề đầu tiên và toàn bộ transcript
                        segments[0] = "\n".join(first_segment_lines)
                        transcript = "\n\n".join(segments)
                        
                    except Exception as e:
                        print(f"    - Could not adjust first subtitle: {e}")

        # ======================================

        adjusted_transcript = adjust_srt_segment(transcript, global_index_offset, start_ms)
        srt_results.append(adjusted_transcript)
        
        num_segments = len([seg for seg in transcript.strip().split('\n\n') if seg.strip() != ""])
        global_index_offset += num_segments

    combined_srt = "\n\n".join(srt_results)
    return combined_srt

# ===== PHẦN THỰC THI CHÍNH (Giữ nguyên) =====
# (Bạn có thể chạy phần này như cũ)

# ===== PHẦN THỰC THI CHÍNH =====
if not gpt_api_key:
    print("Lỗi: Biến môi trường CHATGPT_API_KEY chưa được thiết lập.")
else:
    openai.api_key = gpt_api_key
    print(f"Scanning for .mp3 files in: {input_folder}")

    # Lặp qua tất cả các file trong thư mục đầu vào
    for filename in os.listdir(input_folder):
        if filename.lower().endswith(".mp3"):
            file_path = os.path.join(input_folder, filename)
            print(f"\n{'='*50}\nProcessing file: {file_path}\n{'='*50}")

            try:
                # Gọi hàm xử lý cho từng file audio
                final_srt = process_long_audio(
                    file_path,
                    chunk_length_ms=chunk_length_ms,
                    target_bitrate=target_bitrate
                )

                # Lưu file SRT cuối cùng vào cùng thư mục
                base_name = os.path.splitext(filename)[0]
                output_file = os.path.join(input_folder, f"{base_name}.srt")
                with open(output_file, "w", encoding="utf-8") as f:
                    f.write(final_srt)
                print(f"\nSUCCESS: Full transcription for '{filename}' saved to {output_file}")

            except Exception as e:
                print(f"\nERROR processing file {filename}: {e}")
                print("Skipping to the next file.")
                continue # Bỏ qua file này và tiếp tục với file tiếp theo

    print(f"\n{'='*50}\nAll .mp3 files have been processed.\n{'='*50}")